# Example Notebook for classifier finder

## 0. set log-level of sam_ml library 

(e.g. debugging)

NOTE: has to happen before importing the sam_ml library

In [1]:
import os
os.environ["SAM_ML_LOG_LEVEL"] = "info"

## 1. libraries

In [2]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sam_ml.models import CTest, LR

## 2. data

In [3]:
df = load_iris()
y = pd.Series(df.target)
X = pd.DataFrame(df.data, columns=df.feature_names)
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.80, random_state=42)

## 3. model

### 3.1. create tester class object

CTest is an auto-ml class. You can use it to compare different models and find the best one for your data.

**models**: list of *Classifier* subclass objects or *'all'* (for all integrated wrapper class classifier) or *'basic'* (for a smaller selection of basic classifier)

**vectorizer**, **scaler**, **selector**, **sampler**: CTest init creates *Pipeline* objects out of the given models with the data class parameters given
(look into the *iris_pipeline.ipynb* notebook to see the possible parameters)

In [4]:
tester = CTest("all", scaler="minmax")

get all models in the CTest class object

In [5]:
tester.models

{'LogisticRegression': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=LogisticRegression(random_state=42), model_name='LogisticRegression (pipeline)'),
 'QuadraticDiscriminantAnalysis': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=QuadraticDiscriminantAnalysis(), model_name='QuadraticDiscriminantAnalysis (pipeline)'),
 'LinearDiscriminantAnalysis': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=LinearDiscriminantAnalysis(), model_name='LinearDiscriminantAnalysis (pipeline)'),
 'MLP Classifier': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=MLPClassifier(random_state=42), model_name='MLP Classifier (pipeline)'),
 'Line

you can add models

In [6]:
tester.add_model(LR(model_name="LogisticRegression (elasticnet penalty)", penalty="elasticnet", solver="saga", l1_ratio=0.5))

In [7]:
tester.models

{'LogisticRegression': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=LogisticRegression(random_state=42), model_name='LogisticRegression (pipeline)'),
 'QuadraticDiscriminantAnalysis': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=QuadraticDiscriminantAnalysis(), model_name='QuadraticDiscriminantAnalysis (pipeline)'),
 'LinearDiscriminantAnalysis': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=LinearDiscriminantAnalysis(), model_name='LinearDiscriminantAnalysis (pipeline)'),
 'MLP Classifier': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=MLPClassifier(random_state=42), model_name='MLP Classifier (pipeline)'),
 'Line

you can remove models

In [8]:
tester.remove_model("LogisticRegression (elasticnet penalty)")

In [9]:
tester.models

{'LogisticRegression': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=LogisticRegression(random_state=42), model_name='LogisticRegression (pipeline)'),
 'QuadraticDiscriminantAnalysis': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=QuadraticDiscriminantAnalysis(), model_name='QuadraticDiscriminantAnalysis (pipeline)'),
 'LinearDiscriminantAnalysis': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=LinearDiscriminantAnalysis(), model_name='LinearDiscriminantAnalysis (pipeline)'),
 'MLP Classifier': Pipeline(vectorizer=None, scaler=Scaler(scaler='minmax', clip=False, copy=True, feature_range=(0, 1), ), selector=None, sampler=None, model=MLPClassifier(random_state=42), model_name='MLP Classifier (pipeline)'),
 'Line

## 3.1. evaluation of the models

CTest has 3 ways implemented to evaluate the models. Depending on the dataset you can choose which one to use

### 3.1.1. one-vs-all cross validation

**Concept:**

The model will be trained on all datapoints except one and then tested on this last one. This will be repeated for all datapoints so that we have our predictions for all datapoints.

**Advantage:** optimal use of information for training

**Disadvantage:** long train time

This concept is very useful for small datasets (datapoints < 150) because the long train time is still not too long and especially with a small amount of information for the model, it is important to use all the information one has for the training.

In [10]:
tester.eval_models_cv(X ,y , avg="macro", small_data_eval=True)
tester.output_scores_as_pd(sort_by="recall", console_out=False)

Crossvalidation:   0%|          | 0/21 [00:00<?, ?it/s]

LogisticRegression (pipeline):   0%|          | 0/150 [00:00<?, ?it/s]

QuadraticDiscriminantAnalysis (pipeline):   0%|          | 0/150 [00:00<?, ?it/s]

LinearDiscriminantAnalysis (pipeline):   0%|          | 0/150 [00:00<?, ?it/s]

MLP Classifier (pipeline):   0%|          | 0/150 [00:00<?, ?it/s]

...

BaggingClassifier (DTC based) (pipeline):   0%|          | 0/150 [00:00<?, ?it/s]

BaggingClassifier (RFC based) (pipeline):   0%|          | 0/150 [00:00<?, ?it/s]

BaggingClassifier (mixed based) (pipeline):   0%|          | 0/150 [00:00<?, ?it/s]

XGBClassifier (pipeline):   0%|          | 0/150 [00:00<?, ?it/s]

,accuracy,precision,recall,s_score,l_score,train_score,train_time
LinearDiscriminantAnalysis,0.980000,0.980125,0.980000,9.904373e-01,1.000000,0.980000,0:00:00
QuadraticDiscriminantAnalysis,0.973333,0.973825,0.973333,9.894085e-01,1.000000,0.980045,0:00:00
BaggingClassifier (DTC based),0.960000,0.960000,0.960000,9.874448e-01,1.000000,0.999911,0:00:00
AdaBoostClassifier (RFC based),0.953333,0.953448,0.953333,9.861117e-01,1.000000,1.000000,0:00:00
KNeighborsClassifier,0.953333,0.953448,0.953333,9.861117e-01,1.000000,0.960045,0:00:00
BaggingClassifier (RFC based),0.953333,0.953448,0.953333,9.861117e-01,1.000000,0.988456,0:00:00
GaussianNB,0.953333,0.953448,0.953333,9.861117e-01,1.000000,0.959418,0:00:00
RandomForestClassifier,0.953333,0.953448,0.953333,9.861117e-01,1.000000,1.000000,0:00:00
SupportVectorClassifier,0.953333,0.953448,0.953333,9.861117e-01,1.000000,0.979732,0:00:00
GradientBoostingMachine,0.953333,0.953448,0.953333,9.861117e-01,1.000000,1.000000,0:00:00


### 3.1.2. multiple split crossvalidation

does **cv_num** splits and takes the average values for evaluating the model

In [11]:
tester.eval_models_cv(X, y, avg="macro", small_data_eval=False, cv_num=10)
tester.output_scores_as_pd(sort_by="recall", console_out=False)

Crossvalidation:   0%|          | 0/21 [00:00<?, ?it/s]

,accuracy,precision,recall,s_score,l_score,train_score,train_time
LinearDiscriminantAnalysis,0.966667,0.850000,0.833333,0.698512,0.700000,0.979259,0:00:00
QuadraticDiscriminantAnalysis,0.966667,0.850000,0.833333,0.698512,0.700000,0.982222,0:00:00
SupportVectorClassifier,0.953333,0.800000,0.776667,0.599006,0.600000,0.979259,0:00:00
AdaBoostClassifier (RFC based),0.946667,0.800000,0.773333,0.599004,0.600000,1.000000,0:00:00
RandomForestClassifier,0.946667,0.800000,0.773333,0.599004,0.600000,1.000000,0:00:00
KNeighborsClassifier,0.946667,0.800000,0.773333,0.599005,0.600000,0.964444,0:00:00
ExtraTreesClassifier,0.946667,0.800000,0.773333,0.599005,0.600000,1.000000,0:00:00
GradientBoostingMachine,0.926667,0.800000,0.763333,0.598997,0.600000,1.000000,0:00:00
BaggingClassifier (DTC based),0.920000,0.800000,0.760000,0.598951,0.600000,0.992593,0:00:00
DecisionTreeClassifier,0.953333,0.766667,0.743333,0.499746,0.500000,1.000000,0:00:00


### 3.1.3. evaluate on given train-test-split

sometimes it only makes sense to split a dataset in one way so that cross validation is useless

In [12]:
tester.eval_models(x_train, y_train, x_test, y_test, avg="macro")
tester.output_scores_as_pd(sort_by="recall", console_out=False)

Crossvalidation:   0%|          | 0/21 [00:00<?, ?it/s]

,accuracy,precision,recall,s_score,l_score,train_score,train_time
AdaBoostClassifier (RFC based),1.000000,1.000000,1.000000,0.992600,1.000000,1.000000,0:00:00
GradientBoostingMachine,1.000000,1.000000,1.000000,0.992600,1.000000,1.000000,0:00:00
BaggingClassifier (RFC based),1.000000,1.000000,1.000000,0.992600,1.000000,0.966667,0:00:00
BaggingClassifier (DTC based),1.000000,1.000000,1.000000,0.992600,1.000000,1.000000,0:00:00
GaussianNB,1.000000,1.000000,1.000000,0.992600,1.000000,0.950000,0:00:00
ExtraTreesClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,1.000000,0:00:00
KNeighborsClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,0.958333,0:00:00
AdaBoostClassifier (DTC based),1.000000,1.000000,1.000000,0.992600,1.000000,0.966667,0:00:00
XGBClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,1.000000,0:00:00
SupportVectorClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,0.975000,0:00:00


### 3.2. find best model

**Idea:**

The find_best_model_randomCV method is using randomCVsearch for every modeltype to find its best hyperparameters and afterwards comparing the results of the modeltypes. The randomCVsearch should be just for a few models per modeltype to save time.

**Useful parameters:**

- you can change the used crossvalidation with **small_data_eval** and if *small_data_eval=False*, you can set the number of CVs with **cv_num**

- with the **scoring** parameter you can choose which metric to look at for searching the best model (you can use **avg**, **secondary_scoring**, **strength**, and **pos_label** to more specify it)

- you can change the number of parameter sets to test for each model type with **n_trails**

- you can change with **leave_loadbar** if the loading bar from the  randomCVsearches of each model type shall disappear after they finished

In [13]:
scores = tester.find_best_model_randomCV(x_train, y_train, x_test, y_test, scoring="recall", avg="macro", small_data_eval=False)

randomCVsearch:   0%|          | 0/21 [00:00<?, ?it/s]

randomCVsearch (LogisticRegression (pipeline)):   0%|          | 0/5 [00:00<?, ?it/s]

2023-06-27 12:34:34,878 - sam_ml.models.ClassifierTest - INFO - LogisticRegression (pipeline) - score: 0.9506172839506172 (recall) - parameters: {'C': 63.512210106407046, 'penalty': 'l2', 'solver': 'saga'}


randomCVsearch (QuadraticDiscriminantAnalysis (pipeline)):   0%|          | 0/5 [00:00<?, ?it/s]

2023-06-27 12:34:35,018 - sam_ml.models.ClassifierTest - INFO - QuadraticDiscriminantAnalysis (pipeline) - score: 0.9598765432098766 (recall) - parameters: {'reg_param': 0.0}


...

2023-06-27 12:35:00,466 - sam_ml.models.ClassifierTest - INFO - BaggingClassifier (RFC based) (pipeline) - score: 0.9449955908289241 (recall) - parameters: {'bootstrap': True, 'bootstrap_features': False, 'estimator': RandomForestClassifier(max_depth=4), 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 10}


randomCVsearch (BaggingClassifier (mixed based) (pipeline)):   0%|          | 0/5 [00:00<?, ?it/s]

2023-06-27 12:35:02,549 - sam_ml.models.ClassifierTest - INFO - BaggingClassifier (mixed based) (pipeline) - score: 0.9542548500881834 (recall) - parameters: {'bootstrap': True, 'bootstrap_features': True, 'estimator': KNeighborsClassifier(), 'max_features': 4, 'max_samples': 0.373818018663584, 'n_estimators': 205}


randomCVsearch (XGBClassifier (pipeline)):   0%|          | 0/5 [00:00<?, ?it/s]

2023-06-27 12:35:03,014 - sam_ml.models.ClassifierTest - INFO - XGBClassifier (pipeline) - score: 0.9459876543209876 (recall) - parameters: {'colsample_bytree': 1.0, 'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 0, 'reg_lambda': 1.0}
2023-06-27 12:35:03,083 - sam_ml.models.ClassifierTest - INFO - best model type LogisticRegression - recall: 1.0 - parameters: {'C': 63.512210106407046, 'penalty': 'l2', 'solver': 'saga'}


In [14]:
tester.output_scores_as_pd(sort_by=["recall", "train_time"], console_out=False)

,accuracy,precision,recall,s_score,l_score,train_time,train_score,best_score (rCVs),best_hyperparameters (rCVs)
LogisticRegression,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.966667,0.950617,"{'C': 63.512210106407046, 'penalty': 'l2', 'so..."
LinearDiscriminantAnalysis,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.975000,0.981481,{'solver': 'svd'}
DecisionTreeClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.950000,0.951720,"{'criterion': 'entropy', 'max_depth': 9, 'min_..."
RandomForestClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.966667,0.957341,"{'bootstrap': True, 'criterion': 'gini', 'max_..."
GradientBoostingMachine,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.958333,0.952932,"{'criterion': 'friedman_mse', 'learning_rate':..."
AdaBoostClassifier (RFC based),1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,1.000000,0.952932,"{'algorithm': 'SAMME.R', 'estimator': RandomFo..."
KNeighborsClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,1.000000,0.954255,"{'leaf_size': 19, 'n_neighbors': 5, 'p': 4, 'w..."
ExtraTreesClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.983333,0.952932,"{'bootstrap': False, 'criterion': 'gini', 'max..."
GaussianNB,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.950000,0.938051,{'var_smoothing': 1e-09}
GaussianProcessClassifier,1.000000,1.000000,1.000000,0.992600,1.000000,0:00:00,0.941667,0.932150,"{'max_iter_predict': 47, 'multi_class': 'one_v..."
